In [3]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from collections import defaultdict
import itertools
from urllib.parse import parse_qs
import glob
import matplotlib.pylab as plt
from collections import Counter
import collections
from nltk.tokenize import word_tokenize

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/CSR/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
def get_job_links_page(page):
    base_url = "https://www.indeed.com/jobs?"
    params = {'q': 'data scientist', 
             'l': 'Boulder, CO'}
    

    # start = "https://www.indeed.com/jobs?q=Data+Scientist&l=Denver%2C+CO"
    # use a fake header
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
    params['start'] = 20 * (page-1)

    page = requests.get(base_url, params=params, headers=headers)
    # test = requests.get(start, headers=headers)
    
    soup = BeautifulSoup(page.text, "html.parser")
    links = soup.find_all("a")
    
    # build a list of links
    some_links = []

    for l in links:
        try:
            hyperlink = l.attrs.get('href')
            if "/rc/clk?" in hyperlink:
                some_links.append(l.attrs.get('href'))
        except:
            pass
    
    job_links = ["https://www.indeed.com{}".format(x)
             for x in some_links
             ]
    
    return job_links


In [7]:
links = [get_job_links_page(x) for x in range(1, 40)] # getting a link for 39 pages 

In [9]:
merged = list(itertools.chain(*links))
len(merged)
merged[0] # getting a link for 39 pages 

'https://www.indeed.com/rc/clk?jk=c9d84c941c6b6d2f&fccid=4600afcfb0f58ad6&vjs=3'

In [10]:
def get_filename_from_url(some_url):
    parsed = parse_qs(some_url)
    fccid =  parsed.get('fccid')[0]
    other_id = parsed.get('https://www.indeed.com/rc/clk?jk')[0]
    return fccid+other_id+ ".html"

In [ ]:
get_filename_from_url(merged[0]) # file name for each html file

In [ ]:
len(set([get_filename_from_url(x) for x in merged])) # the total number of html files(jobs) parsed and saved locally. 

In [ ]:
#downloading each html file as a soup object
def download_job_page(link):
    save_name = get_filename_from_url(link)
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36'}
    test = requests.get(link, headers=headers)
    soup = BeautifulSoup(test.text, "html.parser")
    
    with open('data_scienceBoulder_{}'.format(save_name), 'w') as f:
        f.write(str(soup))

In [ ]:
for link in tqdm(merged):
    try:
        download_job_page(link)
    except Exception as e:
        print(str(e), link)
    finally:
        time.sleep(1)


In [2]:
html = glob.glob('*.html')
len(html) # the total number of html files(jobs) parsed and saved locally. 

1775

In [3]:
html_divs = []

for html_file in html:
    with open(html_file, 'r',encoding="ISO-8859-1") as f:
        _data = BeautifulSoup(f.read(), "html.parser")
        try:
            has_span_job_summary = _data.find("span", id="job_summary").get_text()
            html_divs.append(has_span_job_summary)
        except:
            print('The job summary field does not exist.')
        
len(html_divs) # total numbers of job postings saved locally. 

The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.
The job summary field does not exist.


1763

In [4]:
jobs = pd.DataFrame(html_divs)
jobs.columns = ['job_summary']
jobs.head(20)

,job_summary
0,Come join a fast-paced technology organization...
1,Summary\nThe Business Intelligence Analyst is ...
2,Data Administrator\nFlexential stands for some...
3,The Institutional Planning & Analysis departme...
4,Location(s): CO - Denver; MN - Minneapolis\n\n...
5,"Perform business architecture, Human Resources..."
6,"Working as a member of Blue Moon Digital, Inc...."
7,"Dedicated to quality, innovation, engagement, ..."
8,Job Description\n\nRLH Revenue Team Overview:\...
9,Arcadis is looking for an experienced Manageme...


In [ ]:
jobs.shape

In [5]:
df1=pd.read_csv('hacker_news.csv')
del df1['source']
df1.rename(columns={'desc': 'job_summary'}, inplace=True)
df1.head()

,job_summary
0,"Photofeeler (<a href=""https:&#x2F;&#x2F;www.ph..."
1,"Q-Branch Labs (<a href=""https:&#x2F;&#x2F;www...."
2,"Worldwide, REMOTE, full time or part time -- 1..."
3,"Lendable | Senior Software Engineers, Senior D..."
4,"Gambit Research Ltd (<a href=""http:&#x2F;&#x2F..."


In [12]:
all_jobs=pd.concat([df1,jobs])

NameError: name 'df1' is not defined

In [7]:
all_jobs.job_summary.unique()

array(['Photofeeler (<a href="https:&#x2F;&#x2F;www.photofeeler.com" rel="nofollow">https:&#x2F;&#x2F;www.photofeeler.com</a>) | Software Engineers, Marketing Roles | REMOTE or Onsite near Boulder, CO<p>If you were a fan of the old OkCupid blog or Christian Rudder&#x27;s &quot;Dataclysm,&quot; you will love it here.<p>Photofeeler is a tool for photo feedback, but we&#x27;re primarily a data science company. We do math to maximize the statistical value of small, noisy data samples. Our company is changing the future of online profiles.<p>The gig offers unparalleled freedom without beauracratic nonsense (think: can work remote, work flexible hours, no unnecessary meetings). Our users are eager and enthusiastic for everything that we build.<p>More info: <a href="https:&#x2F;&#x2F;www.photofeeler.com&#x2F;jobs" rel="nofollow">https:&#x2F;&#x2F;www.photofeeler.com&#x2F;jobs</a>',
       'Q-Branch Labs (<a href="https:&#x2F;&#x2F;www.q-branch.net" rel="nofollow">https:&#x2F;&#x2F;www.q-branc

In [ ]:
def load_skills_list():
    import pandas as pd
    skills = pd.read_csv('skill_phrases-JBM.csv', encoding="ISO-8859-1")
    skills.columns = ['skill_name']
    skills['skill_name'] = skills.skill_name.map(lambda x: 
                                               x.lower().strip())
    return skills.skill_name.tolist()

In [8]:
skills = pd.read_csv('skill_phrases-JBM.csv', encoding="ISO-8859-1")
skills.columns = ['skill_name']
skills['skill_name'] = skills.skill_name.map(lambda x: 
                                           x.lower().strip())
skills.head()

,skill_name
0,social aptitudes
1,geodatabase
2,e2e
3,analysts
4,hive


In [9]:
list_of_skills = skills.skill_name.tolist() # All skills we know of
list_of_skills = [x.replace("\xa0", " ").replace("\x92s", " ") for x in list_of_skills]

In [11]:
#make a big string of all job descriptions
job_string = " ".join(all_jobs.job_summary.tolist())
job_tokens = nltk.word_tokenize(job_string)
job_tokens = [word for word in job_tokens if len(word) >= 1] # remove space 
job_tokens = [word for word in job_tokens if not word in stop_words] # remove english language stop words 
job_tokens = [word.lower().strip() for word in job_tokens if not word in string.punctuation]

assert(type(job_tokens) == list)
filtered = Counter([x for x in job_tokens if x in list_of_skills])

NameError: name 'all_jobs' is not defined

In [12]:
filtered.most_common(10) # looking across skill frequency for all jobs, one way to do it

[('development', 3445),
 ('software', 3245),
 ('analysis', 3145),
 ('research', 2299),
 ('design', 2008),
 ('engineering', 1983),
 ('communication', 1619),
 ('analyst', 1474),
 ('engineers', 1396),
 ('analytics', 1235)]

In [26]:
def get_matching_skills_for_search_kw(search_term, dataframe_name, list_of_skills):

    
    matches = defaultdict(int)

    for index, row in dataframe_name.iterrows():
        job_desc = row.job_summary # local specificity 
        job_tokens = nltk.word_tokenize(job_desc)
        job_tokens = [word for word in job_tokens if len(word) >= 1] # remove space 
        job_tokens = [word for word in job_tokens if not word in stop_words] # remove english language stop words 
        job_tokens = [word.lower().strip() for word in job_tokens if not word in string.punctuation ] 
        
        if search_term in job_tokens:
            for skill in list_of_skills:
                if (skill in job_tokens) & (skill !=search_keyword):
                    matches[skill] += 1
                    
    return Counter(matches)

In [28]:
LIST_OF_QUERIES = ['sql', 'python', 'javascript', 'java', 'excel'] # obviously use more

all_matches =  { query: get_matching_skills_for_search_kw(query, 
                                                all_jobs, 
                                                list_of_skills) 
               for query in LIST_OF_QUERIES  }

NameError: name 'search_keyword' is not defined

In [ ]:
all_matches
# all_matches ===> {query: counter object, query: counter object}

In [ ]:
counter=collections.Counter(matches)
counter.most_common(5)

In [ ]:
# Pie chart
labels = matches.keys()
sizes = matches.values()
fig1, ax1 = plt.subplots()
fig1.set_size_inches(4,4)
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', pctdistance=0.8, labeldistance=1.3,
        shadow=True, startangle=90)
#plt.title('% of Skills')
centre_circle = plt.Circle((0,0),0.70,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
ax1.axis('equal')  
plt.tight_layout()
plt.show()

In [ ]:
glob.glob('*.csv')

In [ ]:
df1=pd.read_csv('hacker_news.csv')
del df1['source']

In [ ]:
df1.head()

In [ ]:
#del df1['source']
df1.head()

In [ ]:
mark_list = df1.job_summary.tolist()
mark_list[0]

In [ ]:
mark_list = [x.replace('\<a href="https:&#x2F;&#x2F;www.photofeeler.com"', " ").replace("\x92s", " ") for x in mark_list]

In [ ]:
mark_list[0]

In [ ]:
df1.shape

In [ ]:
df1.rename(columns={'desc': 'job_summary'}, inplace=True)
df1.head()

In [ ]:
df_merged=pd.concat([df1,jobs])

In [ ]:
df_merged.head()

In [ ]:
print(df_merged.job_summary.unique().shape)
print(df_merged.shape)

In [ ]:
for index, row in jobs.iterrows():
    
    job_desc = row.job_summary

    #job_tokens = nltk.word_tokenize(job_desc)
    
    job_tokens = [word_tokenize(word) for word in tqdm(job_desc)]
    
    
    #job_tokens = [word for word in job_tokens if len(word) >= 1] # remove space 
    
    #job_tokens = [word for word in job_tokens if not word in stop_words] # remove english language stop words 
    
    #job_tokens = [word.lower().strip() for word 
                  #in job_tokens if not word in string.punctuation ] # 


In [ ]:
len(job_tokens)

In [ ]:
match_found = 0
search_keyword= "data"
matching_skill= search_keyword in job_tokens
    
if matching_skill:
    match_found +=1
    matches= defaultdict(int)  
        
    for skill in list_of_skills:
        if (skill in job_tokens) & (skill !=search_keyword):
            matches[skill] += 1

In [ ]:
match_found